In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from datetime import datetime
import pandas as pd

app = FastAPI()

# =========================
# 1. DataFrames iniciais
# =========================
clientes = pd.DataFrame(columns=["id_cliente", "nome"])
contas = pd.DataFrame(columns=["id_conta", "id_cliente", "saldo"])
transacoes = pd.DataFrame(columns=["id_transacao", "id_conta", "valor", "dataHora", "tipo"])

# =========================
# 2. Modelos de entrada
# =========================
class Cliente(BaseModel):
    id_cliente: int
    nome: str

class Conta(BaseModel):
    id_conta: int
    id_cliente: int
    saldo: float

class Transacao(BaseModel):
    id_conta: int
    valor: float
    dataHora: datetime
    tipo: str   # "CREDITO" ou "DEBITO"

# =========================
# 3. Endpoints
# =========================

# Criar cliente
@app.post("/cliente")
def criar_cliente(cliente: Cliente):
    global clientes
    if cliente.id_cliente in clientes["id_cliente"].values:
        raise HTTPException(status_code=400, detail="Cliente já existe.")
    clientes = pd.concat([clientes, pd.DataFrame([cliente.dict()])], ignore_index=True)
    return {"mensagem": "Cliente criado com sucesso!", "cliente": cliente.dict()}

# Criar conta
@app.post("/conta")
def criar_conta(conta: Conta):
    global contas, clientes
    if conta.id_conta in contas["id_conta"].values:
        raise HTTPException(status_code=400, detail="Conta já existe.")
    if conta.id_cliente not in clientes["id_cliente"].values:
        raise HTTPException(status_code=404, detail="Cliente não encontrado.")
    contas = pd.concat([contas, pd.DataFrame([conta.dict()])], ignore_index=True)
    return {"mensagem": "Conta criada com sucesso!", "conta": conta.dict()}

# Receber transação
@app.post("/transacao")
def receber_transacao(transacao: Transacao):
    global transacoes, contas

    # Verificar se conta existe
    if transacao.id_conta not in contas["id_conta"].values:
        raise HTTPException(status_code=404, detail="Conta não encontrada.")

    # Verificar saldo em caso de débito
    saldo_atual = contas.loc[contas["id_conta"] == transacao.id_conta, "saldo"].values[0]
    if transacao.tipo.upper() == "DEBITO" and saldo_atual + transacao.valor < 0:
        raise HTTPException(status_code=400, detail="Saldo insuficiente para débito.")

    # Registrar transação
    novo_id = len(transacoes) + 1
    nova_transacao = {
        "id_transacao": novo_id,
        "id_conta": transacao.id_conta,
        "valor": transacao.valor,
        "dataHora": transacao.dataHora,
        "tipo": transacao.tipo.upper()
    }
    transacoes = pd.concat([transacoes, pd.DataFrame([nova_transacao])], ignore_index=True)

    # Atualizar saldo
    contas.loc[contas["id_conta"] == transacao.id_conta, "saldo"] += transacao.valor

    return {"mensagem": "Transação registrada com sucesso!", "transacao": nova_transacao}

# Listar clientes
@app.get("/clientes")
def listar_clientes():
    return clientes.to_dict(orient="records")

# Listar contas
@app.get("/contas")
def listar_contas():
    return contas.to_dict(orient="records")

# Relatório de saldos por cliente
@app.get("/relatorio/saldos")
def relatorio_saldos():
    relatorio = contas.merge(clientes, on="id_cliente")
    return relatorio.to_dict(orient="records")

# Relatório de transações por tipo
@app.get("/relatorio/transacoes")
def relatorio_transacoes():
    resumo = transacoes.groupby("tipo")["valor"].sum().reset_index()
    return resumo.to_dict(orient="records")